In [2]:
# conda

!conda install -c esri tensorflow-addons

^C


Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
failed



Examining @/win-64::__win==0=0:   0%|          | 0/5 [00:00<?, ?it/s]
Examining conda-forge/win-64::python==3.10.8=h0269646_0_cpython:  20%|██        | 1/5 [00:00<?, ?it/s]
Examining @/win-64::__archspec==1=x86_64:  80%|████████  | 4/5 [00:00<00:00, 16.46it/s]
                                                                                       

Determining conflicts:   0%|          | 0/5 [00:00<?, ?it/s]
Examining conflict for python tensorflow-addons:   0%|          | 0/5 [00:00<?, ?it/s]
                                                                                      

UnsatisfiableError: The following specifications were found
to be incompatible with the existing python installation in your environment:

Specifications:

  - tensorflow-addons -> python[version='>=3.6,<3.7.0a0|>=3.7,<3.8.0a0|>=3.8,<3.9.0a0|>=3.9,<3.10.0a0']

Your python: conda-forge/win-64::python==3.10.8=h0269646_0_cpython

If python is on the left-most side of the chain, that's the version you've asked for

In [ ]:
# standard Library ---- ---- ---- ----
import random
import os
import sys
print("sys.version :", sys.version)
import glob
from pprint import pprint



# etc ---- ---- ---- ----
import warnings
warnings.filterwarnings(action='ignore') 
import tqdm
# if ("cv2" in sys.modules) == False:
#     !pip install opencv-python
import cv2
print("cv2.__version__ :", cv2.__version__)



# Data ---- ---- ---- ----
import numpy as np
print("np.__version__ :", np.__version__)
import pandas as pd
print("pd.__version__ :", pd.__version__)
import matplotlib.pyplot as plt
import seaborn as sns



# ML, DL ---- ---- ---- ----
import tensorflow as tf
print("tf.__version__ :", tf.__version__)
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())
import torch
print("torch.__version__ :", torch.__version__)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
    print('Not connected to a GPU')
else:
    print(gpu_info)


In [4]:
# def ---- ---- ---- ----
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df



def envPath(dict_path=None):
    import platform
    from pprint import pprint

    global root_path
    global data_path

    if dict_path == None:
        dict_path = {
            "Windows": {"root_path": ".", "data_path": "data"},
            "Google_Colab": {"root_path": "../gdrive/MyDrive", "data_path": "data"},
            "Kaggle_notebook": {"root_path": "..", "data_path": "input"},
        }
    # 추후에 dict_path에 Kaggle_notebook이 없으면 추가하는 코드 추가 요망

    env = platform.uname().system

    if env == "Windows":
        env = env

    elif env == "Linux":
        import sys

        # Google_Colab
        if "google.colab" in sys.modules:
            env = "Google_Colab"
            from google.colab import drive

            drive.mount("/gdrive", force_remount=True)

        # Kaggle_notebook
        else:
            import os

            if list(os.walk("/kaggle/"))[0][1] == ["lib", "input", "working"]:
                env = "Kaggle_notebook"

    root_path = dict_path[env]["root_path"]
    data_path = f"{root_path}/{dict_path[env]['data_path']}"
    print("▣ env :", env)
    print()
    print("▣ platform.uname()", "\n", platform.uname(), sep="")
    print()
    print("▣ data_path", "\n", data_path, sep="")


# path ---- ---- ---- ----
envPath()
train_file = "train.csv"
test_file = "test.csv"

train_path = f"{data_path}/{train_file}"
test_path = f"{data_path}/{test_file}"
print()
print("▣ train_path")
print(train_path)
print()
print("▣ test_path")
print(test_path)

▣ env : Windows

▣ platform.uname()
uname_result(system='Windows', node='Psh-Intel12thI5', release='10', version='10.0.22621', machine='AMD64')

▣ data_path
./data

▣ train_path
./data/train.csv

▣ test_path
./data/test.csv
